In [3]:
import cv2
import os
import numpy as np
from cv2 import dnn_superres




In [3]:
# Create an SR object
def upscale_image(image_name,input_directory,output_directory):
    sr = dnn_superres.DnnSuperResImpl.create()
    # Read image
    image = cv2.imread(os.path.join(input_directory,image_name))
    sr.readModel('LapSRN_x2.pb')  # Choose the upscaling model
    sr.setModel('lapsrn', 2)   #Choose the factor

    # Upscale the image
    result = sr.upsample(image)
    cv2.imwrite(output_directory, result)


In [3]:
def upscale_images_in_directory(input_directory, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through each file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full path for the input and output images
            output_path = os.path.join(output_directory, filename)
            # Upscale the image and get the path to the upscaled image
            upscale_image(filename,input_directory, output_path)


input_directory = "training_data/data V8 buildings+planes/train/images"
output_directory = "upscaled_images"
upscale_images_in_directory(input_directory, output_directory)

In [22]:
# Read the image
image = cv2.imread('image_5 copy.jpeg')
# Define the kernel
sharpen_kernel = np.array([[-1,-1,-1], [-1,10,-1], [-1,-1,-1]])
# Apply the kernel
sharpen = cv2.filter2D(image, -1, sharpen_kernel)
# Save image
cv2.imwrite('sharpened_image.jpg', sharpen)


True

In [ ]:
def apply_blur_to_images(input_dir, output_dir, blur_strength):
    # Check if the input directory exists
    if not os.path.exists(input_dir):
        print(f"Error: Input directory '{input_dir}' not found.")
        return
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory '{output_dir}'.")

    # Get a list of all files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Apply blur to each image and save it to the output directory
    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is not None:
            # Apply blur
            blurred_image = cv2.GaussianBlur(image, (blur_strength, blur_strength), 0)

            # Save the blurred image to the output directory
            output_path = os.path.join(output_dir, f"{image_file}")
            cv2.imwrite(output_path, blurred_image)
            print(f"Blur applied to '{image_file}' and saved as '{output_path}'.")
        else:
            print(f"Error loading '{image_file}'. Skipping.")

input_directory = 'data V8 version 4 copy/train/images'
output_directory = 'blurred_images_v2'
blur_strength = 7  # Adjust the blur strength as needed

apply_blur_to_images(input_directory, output_directory, blur_strength)


In [19]:
from PIL import Image

def split_image(image_path, threshold_width):
    # Open the image
    img = Image.open(image_path)

    # Get the width of the image
    img_width, img_height = img.size

    # Check if the image size is larger than the threshold
    if img_width > threshold_width:
        # Calculate the midpoint
        midpoint = img_width // 2

        # Crop the left half
        left_half = img.crop((0, 0, midpoint, img_height))

        # Crop the right half
        right_half = img.crop((midpoint, 0, img_width, img_height))

        # Save the two halves
        left_half.save("left_half_6.jpg")
        right_half.save("right_half_6.jpg")

        print("Image successfully split into left and right halves.")
    else:
        print("Image does not exceed the threshold width.")

image_path = "test_data/image_6.jpeg"
threshold_width = 800  
split_image(image_path, threshold_width)


Image successfully split into left and right halves.


In [ ]:
def convert_to_grayscale(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory '{output_dir}'.")

    # Get a list of all files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Convert each image to grayscale and save it to the output directory
    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)

        # Convert to grayscale
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Save the grayscale image to the output directory
        output_path = os.path.join(output_dir, f"{image_file}")
        cv2.imwrite(output_path, grayscale_image)


input_directory = 'data V8 version 4 copy/train/images'
output_directory = 'output__grayscale_images_5'
convert_to_grayscale(input_directory, output_directory)

In [4]:
image = cv2.imread('image_4 copy.jpeg')

# define the alpha and beta
alpha = 1.5 # Contrast control
beta = 10 # Brightness control

# Readjust brightness
adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

cv2.imwrite('bright_4.jpeg', adjusted)

True

In [6]:


def replace_class_in_txt(directory, new_class):
    # Iterate through each TXT file in the directory
    for txt_file in os.listdir(directory):
        if txt_file.endswith(".txt"):
            # Read lines from the TXT file
            file_path = os.path.join(directory, txt_file)
            with open(file_path, 'r') as file:
                lines = file.read().split('\n')

            # Replace the class in each line with the new class
            updated_lines = [f"{new_class} {' '.join(line.split()[1:])}" for line in lines]

            # Join the updated lines with newline character
            updated_content = '\n'.join(updated_lines)

            # Write the updated content back to the TXT file
            with open(file_path, 'w') as file:
                file.write(updated_content)


txt_directory = r'C:\Users\Max\Desktop/train/labels'
new_class = 3  # Replace with the desired new class
replace_class_in_txt(txt_directory, new_class)
